# Predicting Flagellate concentration with functional regression based on the oceanographic boxes (spatial means)

## Importing

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xskillscore as xs

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import r_regression

from skfda.representation.grid import FDataGrid
from skfda.ml.clustering import KMeans

from skfda.misc.hat_matrix import LocalLinearRegressionHatMatrix
from skfda.preprocessing.smoothing import KernelSmoother

from skfda.ml.regression import HistoricalLinearRegression

from sklearn.metrics import root_mean_squared_error as rmse

import os
import lzma
import dill

from tqdm import tqdm

import cmocean.cm as cm
import salishsea_tools.viz_tools as sa_vi


## Datasets Preparation

In [ ]:
# Creation of the training - testing datasets

def datasets_preparation(dataset, boxes, regions, name, inputs_names):
    
    indx = np.where((dataset.time_counter.dt.month==2) & (dataset.time_counter.dt.day==29))

    targets = dataset[name].to_numpy().reshape(*dataset[name].to_numpy().shape[:1],-1)

    inputs = []
    for i in inputs_names:
        inputs.append(dataset[i].to_numpy().reshape(*dataset[i].to_numpy().shape[:1],-1))
    inputs = np.array(inputs)

    # Deleting 29 of February
    inputs = np.delete(inputs,indx,axis=1)
    targets = np.delete(targets,indx,axis=0)

    # Splitting in years
    inputs = np.array(np.split(inputs,len(np.unique(dataset.time_counter.dt.year)),axis=1))
    targets = np.array(np.split(targets,len(np.unique(dataset.time_counter.dt.year)),axis=0))

    # Transposing
    inputs = np.transpose(inputs, (1,2,0,3))
    targets = np.transpose(targets, (1,0,2))

    indx = np.where(~np.isnan(targets[0]).any(axis=0))
    inputs = inputs[:,:,:,indx[0]]
    targets = targets[:,:,indx[0]]

    regions = np.ravel(regions)
    regions = regions[indx[0]]

    regions_indiv_t = np.zeros((len(np.unique(dataset.time_counter.dt.dayofyear))-1,len(np.unique(dataset.time_counter.dt.year)),len(boxes)))
    regions_indiv_d = np.zeros((len(inputs_names),len(np.unique(dataset.time_counter.dt.dayofyear))-1,len(np.unique(dataset.time_counter.dt.year)),len(boxes)))

    for j in range (0,len(boxes)):

        regions_indiv_d[:,:,:,j] = np.nanmean(np.where(regions==j, inputs, np.nan),axis=3)
        regions_indiv_t[:,:,j] = np.nanmean(np.where(regions==j, targets, np.nan),axis=2)

    inputs = regions_indiv_d
    targets = regions_indiv_t

    return(inputs, targets)


## File Creation

In [ ]:
def file_creation(path, variable, name):

    temp = variable.to_dataset(name=name)
    temp.to_netcdf(path = path + 'targets_predictions.nc', mode='a', encoding={name:{"zlib": True, "complevel": 9}})
    temp.close()
    

## Regressor

In [ ]:
def regressor (inputs, targets, j, r_inputs, n_intervals, lag):

    # Printing of the correlation coefficients
    temp_inputs = np.reshape(inputs,(len(inputs),inputs.shape[1]*inputs.shape[2]), order='F')
    temp_inputs = temp_inputs.transpose()
    temp_targets = np.reshape(targets, (targets.shape[0]*targets.shape[1]), order='F')

    r_inputs[j] = np.round(r_regression(temp_inputs,temp_targets),2)

    # Scaling the inputs
    scaler_inputs = make_column_transformer((StandardScaler(), np.arange(0,len(inputs))))
    temp_inputs = scaler_inputs.fit_transform(temp_inputs)
    temp_inputs = temp_inputs.transpose()
    inputs = np.reshape(temp_inputs,(len(inputs),inputs.shape[1],inputs.shape[2]), order='F')   
    
    # Scaling the targets
    scaler_targets = StandardScaler()
    temp_targets = np.expand_dims(temp_targets,-1)
    temp_targets = scaler_targets.fit_transform(temp_targets)
    targets = temp_targets.reshape(targets.shape, order='F')

    # Final transformations
    inputs = np.transpose(inputs,axes=(2,1,0))
    targets = targets.transpose()
    inputs = FDataGrid(data_matrix=inputs, grid_points=np.arange(0,len(targets[0])))
    targets = FDataGrid(data_matrix=targets, grid_points=np.arange(0,len(targets[0])))

    # Smoothing
    # targets = targets.to_basis(FourierBasis(n_basis=10))
    kernel_estimator = LocalLinearRegressionHatMatrix(bandwidth=1)
    smoother = KernelSmoother(kernel_estimator=kernel_estimator)
    inputs = smoother.fit_transform(inputs)

    model = HistoricalLinearRegression(n_intervals=n_intervals, lag=lag)
    regr = model.fit(inputs,targets)

    return(regr,scaler_inputs,scaler_targets,smoother,r_inputs)


## Scaling

In [ ]:
def scaling(regr,inputs,scaler_inputs,targets,scaler_targets,smoother):

    # Scaling the inputs
    temp = np.reshape(inputs,(len(inputs),inputs.shape[1]*inputs.shape[2]), order='F')
    temp = temp.transpose()
    temp = scaler_inputs.transform(temp)
    temp = temp.transpose()        
    inputs = np.reshape(temp,(len(inputs),inputs.shape[1],inputs.shape[2]), order='F')
        
    inputs = np.transpose(inputs,axes=(2,1,0))
    inputs = FDataGrid(data_matrix=inputs, grid_points=np.arange(0,len(targets)))

    inputs = smoother.transform(inputs)

    predictions = regr.predict(inputs)

    # Post-processing of predictions
    predictions = np.array(predictions.to_grid(np.arange(0,len(targets))).data_matrix)
    predictions = np.squeeze(predictions,2)

    # Scaling the predictions
    temp = np.reshape(predictions, (targets.shape[0]*targets.shape[1]), order='F')
    temp = np.expand_dims(temp,axis=-1)
    temp = scaler_targets.inverse_transform(temp)
    predictions = temp.reshape(predictions.shape, order='F')
    predictions = predictions.transpose()

    return(predictions)


## Scatter Plot

In [ ]:
def scatter_plot(dates,targets, predictions, name):

    indx = pd.DatetimeIndex(dates[0:75]) # From the first year

    # compute slope m and intercept b
    m, b = np.polyfit(targets, predictions, deg=1)

    fig, ax = plt.subplots()

    scatter = ax.scatter(targets,predictions, s = 10, c= indx.month)

    lims = [np.min([ax.get_xlim(), ax.get_ylim()]),
        np.max([ax.get_xlim(), ax.get_ylim()])]

    # plot fitted y = m*x + b
    ax.axline(xy1=(0, b), slope=m, color='r')

    ax.set_xlabel('targets')
    ax.set_ylabel('predictions')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.set_aspect('equal')
    ax.legend(handles=scatter.legend_elements()[0], labels=['February','March','April'])

    ax.plot(lims, lims,linestyle = '--',color = 'k')

    fig.suptitle(name)

    plt.show()

    return(m)

## Plotting (Mean Values)

In [ ]:
def plotting_mean_values(dates,boxes,targets,predictions,r_train,rms_train,slope_train,category,units,region,boxnames,period,labels):

    years = np.unique(dates.year)

    targets = np.reshape(targets,(targets.shape[0]*targets.shape[1],targets.shape[2]), order = 'F')
    predictions = np.reshape(predictions,(predictions.shape[0]*predictions.shape[1],predictions.shape[2]), order = 'F')
    
    targets_masked = np.ma.array(targets)
    predictions_masked = np.ma.array(predictions)

    for year in years:
        targets_masked[(np.where(dates.year==year)[0][-1])] = np.ma.masked
        predictions_masked[(np.where(dates.year==year)[0][-1])] = np.ma.masked

    for i in range (0,len(boxes)):

        temp = pd.DataFrame(np.vstack((r_train[i],rms_train[i],slope_train[i])).transpose(),index=[boxnames[i]],columns=['r','rms [%]','slope'])
        display(temp)

        fig, ax = plt.subplots(figsize=(19,5))

        test = np.arange(0,len(targets_masked))

        ax.plot(test,targets_masked[:,i], label = 'targets')
        ax.plot(test,predictions_masked[:,i], label = 'predictions')
        
        ticks = np.arange(0,len(years)*len(labels),len(labels)/2)
        ticks = np.int16(ticks)
        labels2 = np.tile(labels,len(years))

        ax.set_xticks(ticks, labels2[ticks])

        ax2 = ax.secondary_xaxis('bottom')
        ax2.set_xticks(ticks=np.arange(0,len(years)*len(labels),len(labels)), labels=years)
        ax2.tick_params(length=0, pad=30)

        ax.fill_between((test[0],test[75]), np.max((targets_masked[:,i],predictions_masked[:,i])), np.min((targets_masked[:,i],predictions_masked[:,i])), alpha=0.3, color='grey')
        ax.fill_between((test[151],test[226]), np.max((targets_masked[:,i],predictions_masked[:,i])),np.min((targets_masked[:,i],predictions_masked[:,i])), alpha=0.3, color='grey')
        ax.xaxis.set_tick_params(labelsize=15)
        ax2.xaxis.set_tick_params(labelsize=15)
        ax.yaxis.set_tick_params(labelsize=15)

        plt.suptitle('Mean Nanoflagellate '+category + ' ' +units + ' ' + period + ' ' + region + boxnames[i], fontsize=18)
        plt.legend()

        plt.show()


## Plotting (Mean Peaks)

In [ ]:
def plotting_mean_peaks(dates,boxes,targets,predictions,category,units,region,boxnames,period,labels):

    years = np.unique(dates.year)

    targets = np.reshape(targets,(targets.shape[0]*targets.shape[1],targets.shape[2]), order = 'F')
    predictions = np.reshape(predictions,(predictions.shape[0]*predictions.shape[1],predictions.shape[2]), order = 'F')
    
    targets_masked = np.ma.array(targets)
    predictions_masked = np.ma.array(predictions)

    for year in years:
      
        targets_masked[(np.where(dates.year==year)[0][-1])] = np.ma.masked
        predictions_masked[(np.where(dates.year==year)[0][-1])] = np.ma.masked

    for i in range (0,len(boxes)):

        fig, ax = plt.subplots(figsize=(19,5))

        ax.plot(targets_masked[:,i], label = 'targets')
        ax.plot(predictions_masked[:,i], label = 'predictions')
        
        ticks = np.arange(0,len(years)*len(labels),len(labels)/2)
        ticks = np.int16(ticks)
        labels2 = np.tile(labels,len(years))

        ax.set_xticks(ticks, labels2[ticks])

        ax2 = ax.secondary_xaxis('bottom')
        ax2.set_xticks(ticks=np.arange(0,len(years)*len(labels),len(labels)), labels=years)
        
        ax2.tick_params(length=0, pad=30)
        plt.suptitle('Mean Flagellate '+category + ' ' +units + ' ' + period +  ' ' + region + ' ' + boxnames[i])
        plt.legend()
        plt.show()


## Plotting (Mean Inputs)

In [ ]:
def plotting_mean_inputs(dates,boxes,inputs,inputs_names,region,boxnames, period):

    years = np.unique(dates.year)
    
    ticks = []
    for i in range (0,inputs.shape[1]*inputs.shape[2],inputs.shape[1]):
        ticks.append(i)

    inputs = np.reshape(inputs,(len(inputs), inputs.shape[1]*inputs.shape[2], inputs.shape[3]), order='F')

    for i in range (0,len(boxes)):

        fig, _ = plt.subplots(figsize=(19,5))

        for j in range (0, len(inputs_names)):

            temp_inputs = np.reshape(inputs,(len(inputs),inputs.shape[1]*inputs.shape[2]), order='F')
            temp_inputs = temp_inputs.transpose()
            scaler_inputs = make_column_transformer((MinMaxScaler(), np.arange(0,len(inputs))))
            temp_inputs = scaler_inputs.fit_transform(temp_inputs)
            temp_inputs = temp_inputs.transpose()
            temp_inputs = np.reshape(temp_inputs,(len(inputs),inputs.shape[1],inputs.shape[2]), order='F')

            inputs_masked = np.ma.array(temp_inputs)
            inputs_masked[:,ticks] = np.ma.masked
           
            plt.plot(inputs_masked[j,:,i], label = inputs_names[j])

        plt.xlabel('Years')
        plt.xticks(ticks,years)
        plt.suptitle('Mean inputs ' + period + ' ' + region + ' ' + boxnames[i])
        plt.legend()

        plt.show()

## Post-Training

In [ ]:
def post_training(dates,boxes,targets,predictions,mean,category,units,region,boxnames,period,labels):

    r_train = np.full(len(boxes),np.nan)
    rms_train = np.full(len(boxes),np.nan)
    slope_train = np.full(len(boxes),np.nan)

    for i in range (0,len(boxes)):

        r_train[i] = np.round(np.corrcoef(np.ravel(targets[:,:,i]),np.ravel(predictions[:,:,i]))[0][1],3)
        rms_train[i] = rmse(np.ravel(targets[:,:,i]),np.ravel(predictions[:,:,i])) / mean[i] * 100
        m,_ = np.polyfit(np.ravel(targets[:,:,i]),np.ravel(predictions[:,:,i]), deg=1)
        slope_train[i] = np.round(m,3)
    
    plotting_mean_values(dates,boxes,targets,predictions,r_train,rms_train,slope_train,category,units,region,boxnames,period,labels)

    return(r_train,rms_train,slope_train)


## Plotting (regions)

In [ ]:
def plot_box(ax, corn, colour):

    ax.plot([corn[2], corn[3], corn[3], corn[2], corn[2]], 
    [corn[0], corn[0], corn[1], corn[1], corn[0]], '-', color=colour)
    

## Initiation

In [ ]:
name = 'Flagellate'
units = '[mmol m-2]'
category = 'biomasses'

filename = '/data/ibougoudis/MOAD/files/inputs/apr_jun.nc'
inputs_names = ['Summation_of_solar_radiation', 'Summation_of_longwave_radiation', 'Mean_air_temperature', 'Mean_pressure', 'Mean_precipitation']

n_intervals = 3

if filename[35:42] == 'apr_jun': # 76 days, 4th period
    if n_intervals==3:
          lags = [25, 25, 25, 25, 25, 25, 25, 25, 50]
    elif n_intervals==4:
        lags = [18.25, 18.25, 37.5, 18.25, 18.25, 18.25, 18.25, 18.25, 18.25] 
    period = '(16 Apr - 30 Jun)'
    id = '4'

elif filename[35:42] == 'may_sep': # 153 days, 5th period
    if n_intervals==3:
        lags = [101.3, 50.6, 50.6, 50.6, 50.6, 101.3, 50.6, 50.6, 50.6]
    elif n_intervals==4:
        lags = [38, 38, 38, 38, 38, 38, 38, 38, 38]
    elif n_intervals==5:
        lags = [30.4, 30.4, 30.4, 30.4, 30.4, 30.4, 30.4, 30.4, 30.4] 
    period = '(01 May - 30 Sep)'
    id = '5'

ds = xr.open_dataset(filename)


## Regions

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 9))
mycmap = cm.deep
mycmap.set_bad('grey')
ax.pcolormesh(ds[name][0], cmap=mycmap)
sa_vi.set_aspect(ax)

SoG_north = [650, 730, 100, 200]
plot_box(ax, SoG_north, 'g')
SoG_center = [450, 550, 200, 300]
plot_box(ax, SoG_center, 'b')
Fraser_plume = [380, 460, 260, 330]
plot_box(ax, Fraser_plume, 'm')
SoG_south = [320, 380, 280, 350]
plot_box(ax, SoG_south, 'k')
Haro_Boundary = [290, 350, 210, 280]
plot_box(ax, Haro_Boundary, 'm')
JdF_west = [250, 425, 25, 125]
plot_box(ax, JdF_west, 'c')
JdF_east = [200, 290, 150, 260]
plot_box(ax, JdF_east, 'w')
PS_all = [0, 200, 80, 320]
plot_box(ax, PS_all, 'm')
PS_main = [20, 150, 200, 280]
plot_box(ax, PS_main, 'r')

boxnames = ['SoG_north','SoG_center','Fraser_plume','SoG_south', 'Haro_Boundary', 'JdF_west', 'JdF_east', 'Puget Sound all', 'PS_main']
fig.legend(boxnames)

boxes = [SoG_north,SoG_center,Fraser_plume,SoG_south,Haro_Boundary,JdF_west,JdF_east,PS_all,PS_main]

regions0 = np.full((len(ds.y),len(ds.x)),np.nan)

for i in range (0, len(boxes)):
    regions0[boxes[i][0]:boxes[i][1], boxes[i][2]:boxes[i][3]] = i

regions0 = xr.DataArray(regions0,dims = ['y','x'])

# # Low resolution
# temp = []

# for i in boxes:
#     temp.append([x//5 for x in i])

# boxes = temp

## Training

In [ ]:
# Low resolution

# ds = ds.isel(y=(np.arange(ds.y[0], ds.y[-1], 5)), 
#     x=(np.arange(ds.x[0], ds.x[-1], 5)))

# regions0 = regions0.isel(y=(np.arange(regions0.y[0], regions0.y[-1], 5)), 
#     x=(np.arange(regions0.x[0], regions0.x[-1], 5)))

dataset = ds.sel(time_counter = slice('2007', '2020'))

labels = np.unique(dataset.time_counter.dt.strftime('%d %b'))
indx_labels = np.argsort(pd.to_datetime(labels, format='%d %b'))
labels = labels[indx_labels]

r_inputs = np.zeros((len(boxnames), len(inputs_names)))

inputs,targets = datasets_preparation(dataset,boxes,regions0,name,inputs_names)

regr_all = []
scaler_inputs_all = []
scaler_targets_all = []
smoother_all = []

predictions = np.full(targets.shape,np.nan)

for i in range (0,len(boxes)):

    inputs2 = inputs[:,:,:,i] # inputs of the i cluster
    targets2 = targets[:,:,i] # targets of the i cluster
    regr, scaler_inputs, scaler_targets, smoother, r_inputs = regressor(inputs2,targets2,i,r_inputs,n_intervals,lags[i])

    scaler_inputs_all.append(scaler_inputs)
    scaler_targets_all.append(scaler_targets)
    smoother_all.append(smoother)
    regr_all.append(regr)

    predictions[:,:,i] = scaling(regr_all[i],inputs2,scaler_inputs_all[i],targets2,scaler_targets_all[i],smoother_all[i]) # putting them in the right place

print('Metrics between input features and '+name)
temp = pd.DataFrame(r_inputs, index=boxnames, columns=inputs_names)
display(temp)


## Heatmaps

In [ ]:
for i in range(0,len(inputs_names)):

    fig, axs = plt.subplots(1,len(boxes), figsize = (28,6), layout='constrained')

    for j in range(0,len(boxes)):

        temp = regr_all[j].coef_
        coeff = temp.data_matrix
        coeff = np.where(coeff==0,np.nan,coeff)

        if j==0: # first time for this input feature

            vmin = np.nanmin(coeff[0,:,:,i])
            vmax = np.nanmax(coeff[0,:,:,i])

        h = axs[j].imshow(coeff[0,:,:,i], cmap='bwr',aspect='auto', vmin=-np.maximum(np.abs(vmin),vmax), vmax=np.maximum(np.abs(vmin),vmax))
        axs[j].set_ylim(axs[j].get_ylim()[::-1])
        cbar = fig.colorbar(h)
        axs[j].set_title('Region ' + boxnames[j])
        axs[j].set_xlabel('Day')
        axs[j].set_ylabel('Day')
        fig.suptitle(inputs_names[i])


## Time-series (Training)

In [ ]:
dates = pd.DatetimeIndex(dataset['time_counter'].values)
indx2 = ~((dataset.time_counter.dt.month==2) & (dataset.time_counter.dt.day==29))
dates = dates[indx2]

r_train,rms_train,slope_train = post_training(dates,boxes,targets,predictions,np.mean(targets,axis=(0,1)),units,category,'(with seasonality)',boxnames, period, labels)

season = np.mean(targets,axis=1)

season_train = np.tile(season,len(np.unique(dates.year))) # Broadcasting season to all training years
season_train = np.reshape(season_train,targets.shape)

plt.plot(season)
plt.xticks(ticks=np.arange(0,len(labels),len(labels)//8+1), labels=labels[np.arange(0,len(labels),len(labels)//8+1)])
plt.suptitle('Long-term seasonality (2007-2020)')
plt.show()

r_train_season,_,slope_train_season = post_training(dates,boxes,targets-season_train,predictions-season_train,np.mean(targets,axis=(0,1)),
    units,category,'(removed seasonality)',boxnames,period,labels)

# Peaks for the testing time-series
peak = np.zeros(len(boxes))
std_targets = np.zeros(len(boxes))
std_season = np.zeros(len(boxes))
std_predictions = np.zeros(len(boxes))

for i in range (0, len(boxes)):
    mean = np.mean(targets[:,:,i])
    std_targets[i] = np.std(targets[:,:,i])
    peak[i] = mean + 0*std_targets[i]
    
    std_season[i] = np.std(season[:,i])
    std_predictions[i] = np.std(predictions[:,:,i])


## Testing years

In [ ]:
dataset = ds.sel(time_counter = slice('2021', '2024'))

dates = pd.DatetimeIndex(dataset['time_counter'].values)
indx = ~((dataset.time_counter.dt.month==2) & (dataset.time_counter.dt.day==29))
dates = dates[indx]

years = np.unique(dataset.time_counter.dt.year)

inputs_test,targets_test= datasets_preparation(dataset,boxes,regions0,name,inputs_names)

season_test = np.tile(season,len(years)) # Broadcasting season to all testing years
season_test = np.reshape(season_test,(targets_test.shape[0],len(years),len(boxes)))

predictions = np.full(targets_test.shape,np.nan)
std_targets_test = np.zeros(len(boxes))
std_predictions_test = np.zeros(len(boxes))

for i in range (0,len(boxes)):

    inputs2 = inputs_test[:,:,:,i] # inputs of the i cluster
    targets2 = targets_test[:,:,i] # targets of the i cluster
    predictions[:,:,i] = scaling(regr_all[i],inputs2,scaler_inputs_all[i],targets2,scaler_targets_all[i],smoother_all[i]) # putting them in the right place

    std_targets_test[i] = np.std(targets2)
    std_predictions_test[i] = np.std(predictions[:,:,i])


## Standard deviations plots

In [ ]:
plt.plot(std_targets)
plt.plot(std_season)
plt.plot(std_targets_test)
plt.plot(std_targets-std_season)
plt.plot(std_targets_test-std_season)

plt.plot(std_predictions)
plt.plot(std_predictions_test)
plt.plot(std_predictions-std_season)
plt.plot(std_predictions_test-std_season)

plt.xticks(ticks=np.arange(0,len(boxes)), labels=boxnames)
plt.xticks(rotation=45)
plt.legend(('targets','season','targets_test','targets-season','targets_test-season','predictions','predictions_test','predictions-season','predictions_test-season'))
plt.suptitle('Standard Deviations for training-seasonanlity-testing')
plt.show()

plt.plot((std_targets_test-std_season)*100/std_targets_test)
plt.xticks(ticks=np.arange(0,len(boxes)), labels=boxnames)
plt.xticks(rotation=45)
plt.suptitle('Percentage of difference between testing')


## Time-series (Testing)

In [ ]:
r_test,rms_test,slope_test = np.zeros(len(boxes)), np.zeros(len(boxes)), np.zeros(len(boxes))

r_test_season, slope_test_season = np.zeros(len(boxes)), np.zeros(len(boxes))

targets_sum, predictions_sum = np.zeros((len(boxes),len(years))), np.zeros((len(boxes),len(years)))

targets_mean, predictions_mean = np.zeros((len(boxes),len(years))), np.zeros((len(boxes),len(years)))

targets_diff, predictions_diff = np.zeros((len(boxes),targets.shape[0],len(years))), np.zeros((len(boxes),targets.shape[0],len(years)))

rss = np.zeros(len(boxes))

r_test_season,_,slope_test_season = post_training(dates,boxes,targets_test,predictions,np.mean(targets_test,axis=(0,1)),
    category,units,'',boxnames,period,labels)

r_test_season,_,slope_test_season = post_training(dates,boxes,targets_test-season_test,predictions-season_test,np.mean(targets_test,axis=(0,1)),
    category,units,'(removed Seasonality) ',boxnames,period,labels)

for i in range (0,len(boxes)):

    r_test[i] = np.round(np.corrcoef(np.ravel(targets_test[:,:,i]),np.ravel(predictions[:,:,i]))[0][1],3)
    rms_test[i] = rmse(np.ravel(targets_test[:,:,i]),np.ravel(predictions[:,:,i])) / np.mean(targets_test[:,:,i]) * 100
    m,_ = np.polyfit(np.ravel(targets_test[:,:,i]),np.ravel(predictions[:,:,i]), deg=1)
    slope_test[i] = np.round(m,3)

    rss[i] = np.sum((np.ravel(targets_test[:,:,i])-np.ravel(predictions[:,:,i]))**2) # Similar to rms, is not affected by the seasonality

    for j in range (0, len(years)):

        targets_sum[i,j] = np.sum(targets_test[:,j,i]-season_test[:,j,i])
        predictions_sum[i,j] = np.sum(predictions[:,j,i]-season_test[:,j,i])

        targets_mean[i,j] = np.mean(targets_test[:,j,i]-season_test[:,j,i])
        predictions_mean[i,j] = np.mean(predictions[:,j,i]-season_test[:,j,i])

        targets_diff[i,:,j] = np.where(targets_test[:,j,i]>peak[i],targets_test[:,j,i], np.nan)
        predictions_diff[i,:,j] = np.where(predictions[:,j,i]>peak[i],predictions[:,j,i], np.nan)

targets_diff = np.transpose(targets_diff,(1,2,0))
predictions_diff = np.transpose(predictions_diff,(1,2,0))
# plotting_mean_peaks(dates,boxes,targets_diff,predictions_diff,category,units,'Peaks',boxnames,period,labels)


## Saving

In [ ]:
# path = '/data/ibougoudis/MOAD/files/results/' + name + '/single_runs/' + name[0:4].lower() + '_func_reg' + id + '_boxes_s8/'

# os.makedirs(path, exist_ok=True)
# with lzma.open(path + 'regr_all.xz', 'wb') as f:  
#     dill.dump(regr_all, f)

# with open(path + 'r_inputs.pkl', 'wb') as f:
#     dill.dump(r_inputs, f)

# with open(path + 'train_metrics.pkl', 'wb') as f:
#     dill.dump([r_train,rms_train,slope_train,r_train_season,slope_train_season,season.transpose()], f)

# with open(path + 'test_metrics.pkl', 'wb') as f:
#     dill.dump([r_test,rms_test,slope_test,r_test_season,slope_test_season,targets_sum,predictions_sum,targets_mean,predictions_mean,targets_diff,predictions_diff,rss], f)

# with open(path + 'targets-predictions.pkl', 'wb') as f:
#     dill.dump([targets_test,predictions], f)

# with open(path + 'readme.txt', 'w') as f:
#     f.write ('name: ' + name)
#     f.write('\n')
#     f.write('period: ' + filename[35:42])
#     f.write ('\n')
#     f.write ('input_features: ')
#     f.write (str([i for i in inputs_names]))
#     f.write ('\n')
#     f.write('n_intervals: ' + str(n_intervals))
#     f.write ('\n')
#     f.write('lags: ')
#     f.write (str([i for i in lags]))
#     f.write ('\n')
